# Setup 

In [1]:
from malariagen_data.pv4 import Pv4
import pandas as pd
import numpy as np
import allel

In [2]:
pv4 = Pv4("gs://pv4_staging/")

In [3]:
pv4_metadata = pv4.sample_metadata()

pv4_metadata.head()

,Sample,Study,Site,First-level administrative division,Country,Lat,Long,Year,ENA,All samples same individual,Population,% callable,QC pass,Exclusion reason,Is returning traveller
0,BBH-1-125,X0009-PV-ET-LO,Jimma,Ethiopia: Oromia,Ethiopia,7.683331,36.851318,2016,ERR2678989,BBH-1-125,AF,88.52,True,Analysis_set,False
1,BBH_1_132,X0009-PV-ET-LO,Jimma,Ethiopia: Oromia,Ethiopia,7.683331,36.851318,2016,ERR2678991,BBH_1_132,AF,90.20,True,Analysis_set,False
2,BBH_1_137,X0009-PV-ET-LO,Jimma,Ethiopia: Oromia,Ethiopia,7.683331,36.851318,2016,ERR2679003,BBH_1_137,AF,87.09,True,Analysis_set,False
3,BBH_1_153,X0009-PV-ET-LO,Jimma,Ethiopia: Oromia,Ethiopia,7.683331,36.851318,2016,ERR2678992,BBH_1_153,AF,90.60,True,Analysis_set,False
4,BBH_1_162,X0009-PV-ET-LO,Jimma,Ethiopia: Oromia,Ethiopia,7.683331,36.851318,2016,ERR2678993,BBH_1_162,AF,91.67,True,Analysis_set,False


## TO DO
Sasha does some filtering of samples to only include 615 high quality monoclonal infection samples (>50% genome positions callable, Fws > 0.95).

Also studies with permission from partner 

# Load variant Data

In [19]:
variant_dataset = pv4.variant_calls(extended=True)
variant_dataset

<xarray.Dataset>
Dimensions:                                   (variants: 4571056, alleles: 7, samples: 1895, ploidy: 2, genotypes: 3, alt_alleles: 6)
Coordinates:
    variant_position                          (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_chrom                             (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    sample_id                                 (samples) object dask.array<chunksize=(1895,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy, genotypes, alt_alleles
Data variables: (12/42)
    variant_allele                            (variants, alleles) object dask.array<chunksize=(65536, 1), meta=np.ndarray>
    variant_filter_pass                       (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_is_snp                            (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_numalt                            (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_CDS                               (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    call_genotype                             (variants, samples, ploidy) int8 dask.array<chunksize=(65536, 64, 2), meta=np.ndarray>
    ...                                        ...
    variant_SNPEFF_IMPACT                     (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_SNPEFF_TRANSCRIPT_ID              (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_SOR                               (variants) float32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_VQSLOD                            (variants) float32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_VariantType                       (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_altlen                            (variants, alt_alleles) int32 dask.array<chunksize=(65536, 6), meta=np.ndarray>

# Load core region data 

Load bed file into pandas dataframe, detailing the chromosome, the start and end, and the type of the region

TODO: Add where file came from - could add to github repo
* same for other things, like FWS...

In [7]:
pv4_regions = pd.read_csv('../supplementary_files/Pv4_regions.bed', sep='\t', comment='t', header=None)
header = ['chrom', 'chromStart', 'chromEnd', 'name']
pv4_regions.columns = header[:len(pv4_regions.columns)]

In [8]:
pv4_regions.loc[pv4_regions.name=='Core'] #Think the vcf is 1 based so might have to shift this by 1/ ask Richard - check with Sasha which one she was using 

,chrom,chromStart,chromEnd,name
1,PvP01_01_v1,116541,677962,Core
3,PvP01_01_v1,679789,903591,Core
6,PvP01_02_v1,100155,162348,Core
8,PvP01_02_v1,164087,745643,Core
11,PvP01_03_v1,108061,630663,Core
13,PvP01_03_v1,632481,894722,Core
16,PvP01_04_v1,185114,564965,Core
18,PvP01_04_v1,566927,685685,Core
20,PvP01_04_v1,748923,967650,Core
23,PvP01_05_v1,143101,844198,Core


# Sliding window through regions 

Subset variants to only include ones which pass filters and are snps

In [52]:
filters = (variant_dataset['variant_filter_pass'].data) & (variant_dataset['variant_is_snp'].data)
filtered_variant_dataset = variant_dataset.isel(variants=filters)
filtered_variant_dataset

<xarray.Dataset>
Dimensions:              (variants: 945649, alleles: 7, samples: 1895, ploidy: 2)
Coordinates:
    variant_position     (variants) int32 dask.array<chunksize=(18192,), meta=np.ndarray>
    variant_chrom        (variants) object dask.array<chunksize=(18192,), meta=np.ndarray>
    sample_id            (samples) object dask.array<chunksize=(1895,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele       (variants, alleles) object dask.array<chunksize=(18192, 1), meta=np.ndarray>
    variant_filter_pass  (variants) bool dask.array<chunksize=(18192,), meta=np.ndarray>
    variant_is_snp       (variants) bool dask.array<chunksize=(18192,), meta=np.ndarray>
    variant_numalt       (variants) int32 dask.array<chunksize=(18192,), meta=np.ndarray>
    variant_CDS          (variants) bool dask.array<chunksize=(18192,), meta=np.ndarray>
    call_genotype        (variants, samples, ploidy) int8 dask.array<chunksize=(18192, 64, 2), meta=np.ndarray>
    call_AD              (variants, samples, alleles) int16 dask.array<chunksize=(18192, 64, 7), meta=np.ndarray>

In [205]:
def get_regions_for_chrom(regions_df, chrom): 
    df = regions_df.loc[(regions_df.chrom == chrom) & (regions_df.name == 'Core')]
    return df 

def count_biallelic_snps_in_window(variant_dataset, window_start, window_end):
    # Subset variants into window 
    variant_dataset_window = variant_dataset.sel(variants=slice(window_start, window_end)) #NEED TO CHECK THIS
    # Count biallelic alleles 
    biallelic_count = count_biallelic_snps(variant_dataset_window)
    return biallelic_count

def count_biallelic_snps(variant_dataset): 
#     should be using variant_numalt instead - check with sasha that everything was on biallelic snps and if so move this to start filtering 
    biallelic_count = (variant_dataset['variant_MULTIALLELIC']=='BI').data.compute().sum()
    # genotypes 
#     gt = allel.GenotypeDaskArray(variant_dataset["call_genotype"].data)
#     # allele counts
#     ac = gt.count_alleles(max_allele=3).compute() #can this change to 2?
#     # Count biallelic 
#     biallelic_count = ac.is_biallelic().sum()
    return biallelic_count

In [204]:
# Set variables 
window_length = 200
step = 50 # change this to step 

In [206]:
# Do this for one chrom just now 
chrom = 'PvP01_01_v1'

# Get core region definitions for chrom 
core_region_df = get_regions_for_chrom(pv4_regions, chrom)
core_region_df

,chrom,chromStart,chromEnd,name
1,PvP01_01_v1,116541,677962,Core
3,PvP01_01_v1,679789,903591,Core


In [207]:
%%time 

# Would this be faster creating df after?
dfObj = pd.DataFrame(columns=['window_start', 'window_end', 'biallelic_count'])

# For each region 
for index, row in core_region_df.iterrows():
    region_start = row.chromStart
    region_end = row.chromEnd
    
    window_start = region_start 
    window_end = window_start + window_length
    
    print(region_start, region_end)
    print('starting iteration')
    
    # Sliding window through region 
    while window_end <= region_end: # should it just be less than? # scikit allel has some functions to do this - pass function, window size and step 
#         https://scikit-allel.readthedocs.io/en/v1.3.3/stats/window.html
        # calculate biallelic snps in window
        biallelic_count = count_biallelic_snps_in_window(filtered_variant_dataset, window_start, window_end)
        
        window_start += step 
        window_end = window_start + window_length
        # Do we need to add a window that's 200 back from the end region so this isn't lost 
        
        dfObj = dfObj.append({'window_start': window_start, 'window_end': window_end, 'biallelic_count': biallelic_count}, 
                             ignore_index=True)
        
dfObj

116541 677962
starting iteration
679789 903591
starting iteration
CPU times: user 8min 21s, sys: 32 s, total: 8min 53s
Wall time: 34min 3s


,window_start,window_end,biallelic_count
0,116591,116791,159
1,116641,116841,156
2,116691,116891,156
3,116741,116941,151
4,116791,116991,171
...,...,...,...
15693,903239,903439,110
15694,903289,903489,122
15695,903339,903539,118
15696,903389,903589,123


In [208]:
def sliding_window_stats(variant_dataset, chrom, region_df, window_length=200, step=50): 
    # Do I add in filtering for pass here? 
    
    # Find core region boundaries for chromosome 
    core_region_df = get_regions_for_chrom(region_df, chrom)
    
    # Set up pandas df for results 
    results_df = pd.DataFrame(columns=['window_start', 'window_end', 'biallelic_count'])
    
    # For each region 
    for index, row in core_region_df.iterrows():
        region_start = row.chromStart
        region_end = row.chromEnd

        window_start = region_start 
        window_end = window_start + window_length

        print(f'starting sliding window for region: {region_start}-{region_end}')

        # Sliding window through region 
        while window_end <= region_end: # should it just be less than? 

            # calculate biallelic snps in window
            biallelic_count = count_biallelic_snps_in_window(variant_dataset, window_start, window_end)

            #shift window
            window_start += step 
            window_end = window_start + window_length
            # Do we need to add a window that's 200 back from the end region so this isn't lost 
            
            # Update results 
            results_df = results_df.append({'window_start': window_start, 'window_end': window_end, 'biallelic_count': biallelic_count}, 
                                 ignore_index=True)
    return results_df

In [210]:
sliding_window_stats(filtered_variant_dataset, 'PvP01_02_v1', pv4_regions, step= 30000)

starting sliding window for region: 100155, 162348
starting sliding window for region: 164087, 745643


,window_start,window_end,biallelic_count
0,130155,130355,124
1,160155,160355,116
2,190155,190355,58
3,194087,194287,123
4,224087,224287,163
5,254087,254287,157
6,284087,284287,140
7,314087,314287,168
8,344087,344287,146
9,374087,374287,182


# Playground

In [29]:
variant_dataset = pv4.variant_calls()
pos = variant_dataset["variant_position"].data
pos[:10].compute()

array([23, 26, 27, 34, 41, 42, 43, 48, 52, 54], dtype=int32)

In [93]:
# Fitler variant dataset to be in window 
windowed_variant_dataset = pass_variant_dataset.sel(variants=slice(window_start, window_end))
windowed_variant_dataset

<xarray.Dataset>
Dimensions:              (variants: 200, alleles: 7, samples: 1895, ploidy: 2)
Coordinates:
    variant_position     (variants) int32 dask.array<chunksize=(200,), meta=np.ndarray>
    variant_chrom        (variants) object dask.array<chunksize=(200,), meta=np.ndarray>
    sample_id            (samples) object dask.array<chunksize=(1895,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele       (variants, alleles) object dask.array<chunksize=(200, 1), meta=np.ndarray>
    variant_filter_pass  (variants) bool dask.array<chunksize=(200,), meta=np.ndarray>
    variant_is_snp       (variants) bool dask.array<chunksize=(200,), meta=np.ndarray>
    variant_numalt       (variants) int32 dask.array<chunksize=(200,), meta=np.ndarray>
    variant_CDS          (variants) bool dask.array<chunksize=(200,), meta=np.ndarray>
    call_genotype        (variants, samples, ploidy) int8 dask.array<chunksize=(200, 64, 2), meta=np.ndarray>
    call_AD              (variants, samples, alleles) int16 dask.array<chunksize=(200, 64, 7), meta=np.ndarray>

In [90]:
%%time 
# genotypes within selected region
gt_region = allel.GenotypeDaskArray(windowed_variant_dataset["call_genotype"].data)
# allele counts
ac_region = gt_region.count_alleles(max_allele=3).compute()
ac_region.is_biallelic().sum()

CPU times: user 2.62 s, sys: 567 ms, total: 3.19 s
Wall time: 9.86 s


156

In [54]:
# In window 116591 to 116791, calculate biallelic snp count 
window_start = 116591
window_end = 116791

# c = variant_dataset["variant_chrom"].data.compute()

# Filter variant dataset to be chrom
# chrom_variant_dataset = pass_variant_dataset.isel(variants=(pass_variant_dataset["variant_chrom"].data == chrom))

In [66]:
# Fitler variant dataset to be in window 
# windowed_variant_dataset = chrom_variant_dataset.isel(variants=
#                                                       (chrom_variant_dataset["variant_position"]>=window_start) & 
#                                                       (chrom_variant_dataset["variant_position"]<window_end))

<xarray.Dataset>
Dimensions:              (variants: 200, alleles: 7, samples: 1895, ploidy: 2)
Coordinates:
    variant_position     (variants) int32 dask.array<chunksize=(200,), meta=np.ndarray>
    variant_chrom        (variants) object dask.array<chunksize=(200,), meta=np.ndarray>
    sample_id            (samples) object dask.array<chunksize=(1895,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele       (variants, alleles) object dask.array<chunksize=(200, 1), meta=np.ndarray>
    variant_filter_pass  (variants) bool dask.array<chunksize=(200,), meta=np.ndarray>
    variant_is_snp       (variants) bool dask.array<chunksize=(200,), meta=np.ndarray>
    variant_numalt       (variants) int32 dask.array<chunksize=(200,), meta=np.ndarray>
    variant_CDS          (variants) bool dask.array<chunksize=(200,), meta=np.ndarray>
    call_genotype        (variants, samples, ploidy) int8 dask.array<chunksize=(200, 64, 2), meta=np.ndarray>
    call_AD              (variants, samples, alleles) int16 dask.array<chunksize=(200, 64, 7), meta=np.ndarray>

In [82]:
import numpy as np 
np.unique(c)

array(['PvP01_01_v1', 'PvP01_02_v1', 'PvP01_03_v1', 'PvP01_04_v1',
       'PvP01_05_v1', 'PvP01_06_v1', 'PvP01_07_v1', 'PvP01_08_v1',
       'PvP01_09_v1', 'PvP01_10_v1', 'PvP01_11_v1', 'PvP01_12_v1',
       'PvP01_13_v1', 'PvP01_14_v1', 'PvP01_API_v1', 'PvP01_MIT_v1',
       'Transfer.PvP01_00_1.final', 'Transfer.PvP01_00_10.final',
       'Transfer.PvP01_00_100.final', 'Transfer.PvP01_00_101.final',
       'Transfer.PvP01_00_102.final', 'Transfer.PvP01_00_103.final',
       'Transfer.PvP01_00_104.final', 'Transfer.PvP01_00_105.final',
       'Transfer.PvP01_00_106.final', 'Transfer.PvP01_00_107.final',
       'Transfer.PvP01_00_108.final', 'Transfer.PvP01_00_109.final',
       'Transfer.PvP01_00_11.final', 'Transfer.PvP01_00_110.final',
       'Transfer.PvP01_00_111.final', 'Transfer.PvP01_00_113.final',
       'Transfer.PvP01_00_114.final', 'Transfer.PvP01_00_115.final',
       'Transfer.PvP01_00_117.final', 'Transfer.PvP01_00_118.final',
       'Transfer.PvP01_00_119.final', 'Trans

In [67]:
pos = variant_dataset["variant_position"].data
p = pos.compute()

In [68]:
p.max()

3182762

In [25]:
alleles = variant_dataset["variant_allele"].data  # dask array
alleles

dask.array<concatenate, shape=(4571056, 7), dtype=object, chunksize=(65536, 6), chunktype=numpy.ndarray>

In [29]:
# Check if they pass 

In [27]:
alleles[:100].compute()

array([['A', 'G', '', '', '', '', ''],
       ['A', 'T', '', '', '', '', ''],
       ['A', 'T', '', '', '', '', ''],
       ['T', 'G', '', '', '', '', ''],
       ['A', 'T', '', '', '', '', ''],
       ['T', 'A', '', '', '', '', ''],
       ['T', 'G', '', '', '', '', ''],
       ['A', 'G', '', '', '', '', ''],
       ['G', 'T', '', '', '', '', ''],
       ['T', 'A', '', '', '', '', ''],
       ['T', 'A', '', '', '', '', ''],
       ['A', 'C', '', '', '', '', ''],
       ['CA', 'C', 'TA', '', '', '', ''],
       ['A', 'C', '', '', '', '', ''],
       ['C', 'A', '', '', '', '', ''],
       ['A', 'AAT', '', '', '', '', ''],
       ['TAAA', 'T', 'AAAA', 'TAAAA', '', '', ''],
       ['A', 'T', '*', '', '', '', ''],
       ['A', 'T', '', '', '', '', ''],
       ['A', 'G', '', '', '', '', ''],
       ['A', 'T', '', '', '', '', ''],
       ['A', 'T', 'G', '', '', '', ''],
       ['A', 'C', 'T', '', '', '', ''],
       ['T', 'G', '', '', '', '', ''],
       ['T', 'A', '', '', '', '', ''],
     